# SENTIMENT ANALYSIS USING ANY NETWORK AND ANY DATASET:

DATASET - IMDB REVIEW DATASET

Importing basic libraries

Defining function to preprocess data to remove all the punctuations and remove words with 5 or fewer occurences from each review

In [1]:
import numpy as np
import tensorflow as tf

# Preprocessing dataset

Importing dataset and preprocessing it. In preprocesing, we extract the words from each review and remove punctuations.

In [2]:
import re
from collections import Counter
from nltk.corpus import stopwords

def preprocess(text):
    
    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('.', ' <PERIOD> ')
    text = text.replace(',', ' <PERIOD> ')
    text = text.replace('"', ' <PERIOD> ')
    text = text.replace(';', ' <PERIOD> ')
    text = text.replace('!', ' <PERIOD> ')
    text = text.replace('?', ' <PERIOD> ')
    text = text.replace('(', ' <PERIOD> ')
    text = text.replace(')', ' <PERIOD> ')
    text = text.replace('--', ' <PERIOD> ')
    text = text.replace('?', ' <PERIOD> ')
    text = text.replace('<br />', ' <PERIOD> ')
    text = text.replace('\\', ' <PERIOD> ')
    # text = text.replace('\n', ' <NEW_LINE> ')
    text = text.replace(':', ' <PERIOD> ')
    text = text.replace(' <PERIOD> ', ' ')
    words = text.split()
    
    return words

def removing_noise(words):
    word_count = Counter(words)
    stops = set(stopwords.words("english"))
    words_new = [word for word in words if (word_count[word]>5) #and (not word in stops)
                ]
    return words_new
    
    

In [3]:
import csv

filename = '/home/drishti/dhruv/milestone1/data/labeledTrainData.tsv'
review_ids = []
reviews = []
labels = []
#importing dataset into lists
with open(filename, 'r') as f:
    next(f)
    reader = csv.reader(f, delimiter='\t')
    
    for row in reader:
        review_ids.append(row[0])
        labels.append(row[1])
        reviews.append(row[2])

In [4]:
len(reviews)

25000

In [5]:
reviews[0]

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [6]:
labels[:10]

['1', '1', '0', '0', '1', '1', '0', '0', '0', '1']

In [7]:
reviews_pp = []
words = []

for review in reviews:
    review_pp = preprocess(review)
    reviews_pp.append(review_pp)
    words.extend(review_pp)
    
words = removing_noise(words)

## Encoding the words to generate a map for word to integer mapping

Generating a map to convert each word to an integer. The integer assigned to a word is decided by the no. of times it occurs. Higher the occurences, lower is the integer assigned.
After that, each review is converted in the form of integers to feed them well in the network.

In [8]:
#Converting word to integers and making the vocabulary
vocab = set(words)
vocab_size = len(vocab)
words_count = Counter(words)
sorted_vocab = sorted(words_count, key = words_count.get, reverse = True)
word_to_int = {word:i for i,word in enumerate(sorted_vocab)}

#Converting each review in the form of integers
reviews_pp_ints = []
for review in reviews_pp:
    this_review_int = []
    for word in review:
        if word in vocab:
            this_review_int.append(word_to_int[word])
    reviews_pp_ints.append(this_review_int)

In [9]:
max_l = 0
min_l = 10000
for review in reviews_pp_ints:
    if len(review) > max_l:
        max_l = len(review)
    if len(review) < min_l:
        min_l = len(review)

min_l, max_l

(10, 2354)

## Function to make the input layer for the review passed as the argument

This function generates a vector of length same as the vocabulary. Each index corresponds to the equivalent word. The value at that index corresponds to the frequency of that word in the review.

In [20]:
def generate_input_layer(review_int):
    input_layer = np.zeros((1, vocab_size))
    for word in review_int:
        input_layer[0][word] += 1
        
    return input_layer

In [23]:
input_layer = generate_input_layer(reviews_pp_ints[0])
np.shape(input_layer)

(1, 28260)